In [7]:
import os
import smtplib
from email.message import EmailMessage

In [8]:
HOST_EMAIL = os.environ.get('PS5TRACKER')
HOST_EMAIL_PWD = os.environ.get('PS5TRACKERPWD')

In [10]:
def send_warning(web):
    '''
        Send instock warning email to end-user
    '''
    msg = EmailMessage()
    msg['Subject'] = f'Play Station 5 Instock at {web}'
    msg['From'] = HOST_EMAIL
    msg['To'] = HOST_EMAIL
    msg.set_content(f'{web} has PS5 in stock. RUSH!!!!!')
    
    with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
        smtp.ehlo()
        smtp.starttls()
        smtp.ehlo()
        
        smtp.login(HOST_EMAIL, HOST_EMAIL_PWD)
        
        smtp.send_message(msg)